In [1]:
ls

drive/  sample_data/


In [2]:
cd drive/MyDrive/CANDEV2022

/content/drive/MyDrive/CANDEV2022


In [3]:
import pandas as pd
incidents = pd.read_csv('INCIDENTS.csv')
incidents_history = pd.read_csv('INCIDENT_HISTORY.csv')
incidents_owner_history = pd.read_csv('INCIDENT_OWNER_HISTORY.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (12) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [4]:
# delete all datapoints without ticket number
incidents = incidents[~incidents['TICKET_NMBR'].isnull()]
incidents_history = incidents_history[~incidents_history['ticket_nmbr'].isnull()]
incidents_owner_history = incidents_owner_history[~incidents_owner_history['ticket_nmbr'].isnull()]

In [5]:
# Closed status is not important, delete it
incidents_owner_history = incidents_owner_history[incidents_owner_history.STATUS != 'CLOSED']

In [6]:
incidents_owner_history['assigned_group']=incidents_owner_history['assigned_group'].astype('str')

In [7]:
# delete the datapoints without being assigned
incidents_owner_history= incidents_owner_history[incidents_owner_history.assigned_group != 'nan']

In [8]:
# group the owner dataset by ticketnumber, assignedgroup and status
import numpy as np
f = {'assigned_group': lambda x: np.unique(x).tolist(),
     'STATUS':lambda x: np.unique(x).tolist(),
     }
group = incidents_owner_history.groupby('ticket_nmbr', as_index=False).agg(f).reindex(columns=incidents_owner_history.columns)
group = group[['ticket_nmbr', 'assigned_group', 'STATUS']]
# tickets being reassigned to more than two groups
reassigned = group.loc[group.assigned_group.str.len() > 1].reset_index(drop=True)

In [9]:
print("Not reassigned:", len(group) - len(reassigned))
print("Reassigned:", len(reassigned))

Not reassigned: 166850
Reassigned: 58639


In [14]:
reassigned['reassignment_num'] = reassigned.assigned_group.str.len()-1

In [15]:
reassigned

,ticket_nmbr,assigned_group,STATUS,reassignment_num
0,IN10028618,"[DC000152, EDC00029, ESI00011, ESI00045]","[INPROG, QUEUED, RESOLVED]",3
1,IN10028628,"[DC000152, EDC00029]","[QUEUED, RESOLVED]",1
2,IN10028629,"[DC000113, DC000115]","[QUEUED, RESOLVED]",1
3,IN10028648,"[NW000403, NW000417]","[QUEUED, RESOLVED]",1
4,IN10028651,"[EDC00002, ESI00043]","[QUEUED, RESOLVED]",1
...,...,...,...,...
58634,IN10293533,"[DC000203, DC000236, ESI00011]","[QUEUED, RESOLVED]",2
58635,IN10293538,"[DC000152, SM000562]","[INPROG, QUEUED, RESOLVED]",1
58636,IN10293549,"[ESI00011, ITS00379]","[QUEUED, RESOLVED]",1
58637,IN10293552,"[DC000155, SM000562]","[INPROG, QUEUED, RESOLVED]",1


In [16]:
# find the attributes of the tickets being reassigned
data = pd.merge(reassigned, incidents_owner_history, on=['ticket_nmbr'])

In [17]:
data.to_csv('reassignment.csv')